In [1]:
from cim.loaders import Parameter, ConnectionParameters
from cim.loaders.blazegraph.blazegraph import BlazegraphConnection
from cim.loaders.gridappsd import GridappsdConnection, get_topology_response
from cim.models.distributed_model import DistributedModel
import cim.data_profile as cim
import json


In [2]:
# Blazegraph connection for running inside the container
params = ConnectionParameters([Parameter(key="url", value="http://blazegraph:8080/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params)

In [3]:
# Blazegraph connection for running outside the container
params = ConnectionParameters([Parameter(key="url", value="http://localhost:8889/bigdata/namespace/kb/sparql")])
bg = BlazegraphConnection(params)

In [4]:
#feeder_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"  # 123 bus
feeder_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
# feeder_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" # 13 bus asets
#feeder_mrid = "_4F76A5F9-271D-9EB8-5E31-AA362D86F2C3"  # 8500 node
#feeder_mrid = "_67AB291F-DCCD-31B7-B499-338206B9828F" # J1
#feeder_mrid = "_9CE150A8-8CC5-A0F9-B67E-BBD8C79D3095"  # R2 12.47 3
#feeder_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #9500 node

In [5]:
feeder = cim.Feeder(mRID=feeder_mrid)

In [6]:
# topology_response = get_topology_response(feeder_mrid)
with open('topo_message.json') as topo_file:
    topology_response = json.load(topo_file)


In [7]:
network = DistributedModel(connection=bg, feeder=feeder, topology=topology_response)

LoadBreakSwitch
Fuse
LoadBreakSwitch
Breaker
Recloser
ACLineSegment
TransformerTank
ACLineSegment
PowerTransformer
PowerTransformer
TransformerTank
ACLineSegment
ACLineSegment
ACLineSegment
ACLineSegment
ACLineSegment
ACLineSegment
ACLineSegment
TransformerTank
ACLineSegment
TransformerTank
ACLineSegment
ConnectivityNode
ConnectivityNode
ConnectivityNode
ACLineSegment
ACLineSegment
TransformerTank
ACLineSegment
TransformerTank
TransformerTank
ACLineSegment
ConnectivityNode


Breaker
EnergyConsumer
EnergyConsumer
ACLineSegment
ACLineSegment
ACLineSegment
ACLineSegment
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
TransformerTank
PowerTransformer
ConnectivityNode
Fuse
EnergyConsumer
PowerElectronicsConnection
EnergyConsumer
EnergyConsumer
PowerElectronicsConnection
ConnectivityNode
PowerTransformer
LinearShuntCompensator
ACLineSegment
ConnectivityNode
ConnectivityNode
LoadBreakSwitch
LinearShuntCompensator
ACLineSegment
ConnectivityNode
ConnectivityNode
LoadBreakSwitch
TransformerTank
ACLineSegment
ACLineSegment
ACLineSegment
ACLineSegment
ACLineSegment
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
ConnectivityNode
LoadBreakSwitch
LoadBreakSwitch
Recloser
EnergyConsumer
PowerElectronicsConnection
PowerElectronicsConnection
ConnectivityNode
TransformerTank


In [8]:
# network.switch_areas[0].get_all_attributes(cim.LinearShuntCompensator)
network.switch_areas[0].get_all_attributes(cim.ACLineSegment)
network.switch_areas[0].get_all_attributes(cim.WireSpacingInfo)



In [9]:
network.get_all_attributes(cim.ACLineSegment)
network.get_all_attributes(cim.WireSpacingInfo)
network.get_all_attributes(cim.WirePosition)

In [10]:
network.typed_catalog[cim.ACLineSegment]['_0BBD0EA3-F665-465B-86FD-FC8B8466AD53']

ACLineSegment(mRID='_0BBD0EA3-F665-465B-86FD-FC8B8466AD53', aliasName=None, description=None, name='645646', Names=[], AssetDatasheet=None, Assets=[], Controls=[], Location=[Location(mRID='_89C21411-D107-454A-93B0-0905893DD27D', aliasName=None, description=None, name=None, Names=[], Assets=[], CoordinateSystem=None, Fault=[], Measurements=[], PowerSystemResources=[])], Measurements=[Measurement(mRID='_2d9ed148-d492-4e7d-b9af-81a407e74a59', aliasName=None, description=None, name=None, Names=[], measurementType=None, phases=None, Asset=None, Locations=[], PowerSystemResource=None, Terminal=None), Measurement(mRID='_4e30ae83-418c-4cfb-99eb-5572e00b4ec4', aliasName=None, description=None, name=None, Names=[], measurementType=None, phases=None, Asset=None, Locations=[], PowerSystemResource=None, Terminal=None), Measurement(mRID='_5f1dfa9f-e754-471d-9c74-c1a9441e95a3', aliasName=None, description=None, name=None, Names=[], measurementType=None, phases=None, Asset=None, Locations=[], PowerSys

In [11]:
network.typed_catalog[cim.WireSpacingInfo]['_E804FC3A-4DE8-4497-B83D-EBE8974C791B']

KeyError: <class 'cim.data_profile.gridappsd_cim_profile.WireSpacingInfo'>

In [ ]:
from typing import List
import re
a = network.typed_catalog[cim.WireSpacingInfo]['_E804FC3A-4DE8-4497-B83D-EBE8974C791B']
at_type = a.__dataclass_fields__['ACLineSegments'].type
print(at_type)
if 'List' in at_type:
    at_cls = re.match(r'List\[\'(.*)\']',at_type)
    print(at_cls.group(1))
    # re.match(r'<img.*?>',line)
# imgtag.group(0)


In [ ]:
#ridAPPSD Agent Side
bg.create_default_instances(feeder.mRID, network.typed_catalog[cim.WireSpacingInfo]['_E804FC3A-4DE8-4497-B83D-EBE8974C791B'].WirePositions)
network.get_all_attributes(cim.WirePosition)
network.typed_catalog[cim.ACLineSegment]

In [ ]:
json.loads(network.__dumps__(cim.ACLineSegment))


In [12]:
def pretty_print(clas, indent=0):
    print(' ' * indent +  type(clas).__name__ +  ':')
    indent += 4
    for k,v in clas.__dict__.items():
        if '__dict__' in dir(v):
            pretty_print(v,indent)
        else:
            print(' ' * indent +  k + ': ' + str(v))

In [13]:
pretty_print(network.typed_catalog[cim.ACLineSegment]['_0BBD0EA3-F665-465B-86FD-FC8B8466AD53'])

ACLineSegment:
    mRID: _0BBD0EA3-F665-465B-86FD-FC8B8466AD53
    aliasName: None
    description: None
    name: 645646
    Names: []
    AssetDatasheet: None
    Assets: []
    Controls: []
    Location: [Location(mRID='_89C21411-D107-454A-93B0-0905893DD27D', aliasName=None, description=None, name=None, Names=[], Assets=[], CoordinateSystem=None, Fault=[], Measurements=[], PowerSystemResources=[])]
    Measurements: [Measurement(mRID='_2d9ed148-d492-4e7d-b9af-81a407e74a59', aliasName=None, description=None, name=None, Names=[], measurementType=None, phases=None, Asset=None, Locations=[], PowerSystemResource=None, Terminal=None), Measurement(mRID='_4e30ae83-418c-4cfb-99eb-5572e00b4ec4', aliasName=None, description=None, name=None, Names=[], measurementType=None, phases=None, Asset=None, Locations=[], PowerSystemResource=None, Terminal=None), Measurement(mRID='_5f1dfa9f-e754-471d-9c74-c1a9441e95a3', aliasName=None, description=None, name=None, Names=[], measurementType=None, phases=No

In [ ]:
value = 'abc'
separator = ';'
values = value.split(separator)
obj_list = []
obj_list.extend(values)
obj_list

In [15]:
network.typed_catalog[cim.PerLengthImpedance]

{'_EAEF5A85-027A-4BE2-9A07-99D4550C3DE1': PerLengthImpedance(mRID='_EAEF5A85-027A-4BE2-9A07-99D4550C3DE1', aliasName=None, description=None, name=None, Names=[], WireAssemblyInfo=None, ACLineSegments=[]),
 '_12C526C5-6645-4C0C-83CD-3030C440DA17': PerLengthImpedance(mRID='_12C526C5-6645-4C0C-83CD-3030C440DA17', aliasName=None, description=None, name=None, Names=[], WireAssemblyInfo=None, ACLineSegments=[]),
 '_50044320-7295-4591-B39E-F793F1923098': PerLengthImpedance(mRID='_50044320-7295-4591-B39E-F793F1923098', aliasName=None, description=None, name=None, Names=[], WireAssemblyInfo=None, ACLineSegments=[]),
 '_D3933951-0672-4817-B38D-E51F47E8631F': PerLengthImpedance(mRID='_D3933951-0672-4817-B38D-E51F47E8631F', aliasName=None, description=None, name=None, Names=[], WireAssemblyInfo=None, ACLineSegments=[]),
 '_B10FD3C7-8BD6-45D0-9A08-8ADFE7A51952': PerLengthImpedance(mRID='_B10FD3C7-8BD6-45D0-9A08-8ADFE7A51952', aliasName=None, description=None, name=None, Names=[], WireAssemblyInfo=N